In [1]:
# from board import Board
from board_state import BoardState
from bitboard_utils import *
from constants import *
import chess, time
from generate_moves import generate_pseudo_legal_moves, make_move, generate_legal_moves
from perft_test import perft_main, debug_perft
from move import encode_move, get_move_uci, print_move
# from tables import *
import numpy as np

%load_ext autoreload
%autoreload 2

C:\Users\Bastian\Documents\Python\JustAnotherChessEngine\tables.py:275: RuntimeWarning: overflow encountered in ulonglong_scalars
  magic_index = (occupancy * bishop_magic_numbers[sq]) >> np.ulonglong(64 - bishop_relevant_bits[sq])
C:\Users\Bastian\Documents\Python\JustAnotherChessEngine\tables.py:280: RuntimeWarning: overflow encountered in ulonglong_scalars
  magic_index = int((occupancy * rook_magic_numbers[sq]) >> np.ulonglong(64 - rook_relevant_bits[sq]))


In [2]:
fen_list = ["r3k2r/p1ppqpb1/bn2pnp1/3PN3/1p2P3/2N2Q1p/PPPBBPPP/R3K2R w KQkq - ",
            "8/2p5/3p4/KP5r/1R3p1k/8/4P1P1/8 w - - ",
            "r3k2r/Pppp1ppp/1b3nbN/nP6/BBP1P3/q4N2/Pp1P2PP/R2Q1RK1 w kq - 0 1",
            "r2q1rk1/pP1p2pp/Q4n2/bbp1p3/Np6/1B3NBn/pPPP1PPP/R3K2R b KQ - 0 1 ",
            "rnbq1k1r/pp1Pbppp/2p5/8/2B5/8/PPP1NnPP/RNBQK2R w KQ - 1 8  ",
            "r4rk1/1pp1qppp/p1np1n2/2b1p1B1/2B1P1b1/P1NP1N2/1PP1QPPP/R4RK1 w - - 0 10 "]
            
            

In [3]:
#--Illegal ep move #1

fen_list.append("3k4/3p4/8/K1P4r/8/8/8/8 b - - 0 1")# perft 6 = 1134888

#--Illegal ep move #2

fen_list.append("8/8/4k3/8/2p5/8/B2P2K1/8 w - - 0 1")# perft 6 = 1015133

#--EP Capture Checks Opponent

fen_list.append("8/8/1k6/2b5/2pP4/8/5K2/8 b - d3 0 1")# perft 6 = 1440467

#--Short Castling Gives Check

fen_list.append("5k2/8/8/8/8/8/8/4K2R w K - 0 1")# perft 6 = 661072

#--Long Castling Gives Check

fen_list.append("3k4/8/8/8/8/8/8/R3K3 w Q - 0 1")# perft 6 = 803711

#--Castle Rights

fen_list.append("r3k2r/1b4bq/8/8/8/8/7B/R3K2R w KQkq - 0 1")# perft 4 = 1274206

#--Castling Prevented

fen_list.append("r3k2r/8/3Q4/8/8/5q2/8/R3K2R b KQkq - 0 1")# perft 4 = 1720476

#--Promote out of Check

fen_list.append("2K2r2/4P3/8/8/8/8/8/3k4 w - - 0 1")# perft 6 = 3821001

#--Discovered Check

fen_list.append("8/8/1P2K3/8/2n5/1q6/8/5k2 b - - 0 1")# perft 5 = 1004658

#--Promote to give check

fen_list.append("4k3/1P6/8/8/8/8/K7/8 w - - 0 1")# perft 6 = 217342

#--Under Promote to give check

fen_list.append("8/P1k5/K7/8/8/8/8/8 w - - 0 1")# perft 6 = 92683

#--Self Stalemate

fen_list.append("K1k5/8/P7/8/8/8/8/8 w - - 0 1")# perft 6 = 2217

#--Stalemate & Checkmate

fen_list.append("8/k1P5/8/1K6/8/8/8/8 w - - 0 1")# perft 7 = 567584    

#--Stalemate & Checkmate

fen_list.append("8/8/2k5/5q2/5n2/8/5K2/8 b - - 0 1")# perft 4 = 23527

In [4]:
board = BoardState()
b = chess.Board("8/2p5/3p4/KP5r/1R3p1k/8/4P1P1/8 w - - ")
board.set_fen("8/2p5/3p4/KP5r/1R3p1k/8/4P1P1/8 w - - ")
print(board)


8  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ♙ ·  ·  ·  ·  · 
6  ·  ·  ·  ♙ ·  ·  ·  · 
5  ♚ ♟ ·  ·  ·  ·  ·  ♖
4  ·  ♜ ·  ·  ·  ♙ ·  ♔
3  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ♟ ·  ♟ · 
1  ·  ·  ·  ·  ·  ·  ·  · 
   A  B  C  D  E  F  G  H
Side:     
0
Enpassant:   
no 
Castling:    
 


In [6]:
for fen in fen_list:
    board.set_fen(fen)
    print(board)
    b = chess.Board(fen)
    s = time.time()
    r = debug_perft(board, 4, b)
    print("Nodes looked up: ", r)
    t = time.time()
    print("Nodes/s: ", r/(t-s))
    print("time: ", t-s)


8  ♖ ·  ·  ·  ♔ ·  ·  ♖
7  ♙ ·  ♙ ♙ ♕ ♙ ♗ · 
6  ♗ ♘ ·  ·  ♙ ♘ ♙ · 
5  ·  ·  ·  ♟ ♞ ·  ·  · 
4  ·  ♙ ·  ·  ♟ ·  ·  · 
3  ·  ·  ♞ ·  ·  ♛ ·  ♙
2  ♟ ♟ ♟ ♝ ♝ ♟ ♟ ♟
1  ♜ ·  ·  ·  ♚ ·  ·  ♜
   A  B  C  D  E  F  G  H
Side:     
0
Enpassant:   
no 
Castling:    
KQkq 
Nodes looked up:  4085603
Nodes/s:  16503.68387924661
time:  247.55703210830688

8  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ♙ ·  ·  ·  ·  · 
6  ·  ·  ·  ♙ ·  ·  ·  · 
5  ♚ ♟ ·  ·  ·  ·  ·  ♖
4  ·  ♜ ·  ·  ·  ♙ ·  ♔
3  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ♟ ·  ♟ · 
1  ·  ·  ·  ·  ·  ·  ·  · 
   A  B  C  D  E  F  G  H
Side:     
0
Enpassant:   
no 
Castling:    
 
Nodes looked up:  43238
Nodes/s:  11522.737045211255
time:  3.7524070739746094

8  ♖ ·  ·  ·  ♔ ·  ·  ♖
7  ♟ ♙ ♙ ♙ ·  ♙ ♙ ♙
6  ·  ♗ ·  ·  ·  ♘ ♗ ♞
5  ♘ ♟ ·  ·  ·  ·  ·  · 
4  ♝ ♝ ♟ ·  ♟ ·  ·  · 
3  ♕ ·  ·  ·  ·  ♞ ·  · 
2  ♟ ♙ ·  ♟ ·  ·  ♟ ♟
1  ♜ ·  ·  ♛ ·  ♜ ♚ · 
   A  B  C  D  E  F  G  H
Side:     
0
Enpassant:   
no 
Castling:    
kq 
Nodes looked up:  422333
Nodes/s:  163